In [1]:
# -*- coding: utf-8 -*-
"""deep_distance_estimatorPooja.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wD7Z_pun1_dkbuxyEwfZxy14pO5Zvgoa

Code from Rienke on Github, from repository single_cell_segmentation
- only edited to try and work with data from NeuroToolBox Gong Lab

Deep Distance Estimator is trained to learn EDT.
EDT is the euclidean distance transform and emasures the distance between
two pixels


a deep distance estimator is trained to learn EDT. Since the Euclidean distance assumes a continuous instead of a Boolean value, pixels that are close to cell boundary have smaller values. Specifically consider the regions with blurred boundaries in the original images.

MUST use conda environment
"""


'deep_distance_estimatorPooja.ipynb\n\nAutomatically generated by Colaboratory.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1wD7Z_pun1_dkbuxyEwfZxy14pO5Zvgoa\n\nCode from Rienke on Github, from repository single_cell_segmentation\n- only edited to try and work with data from NeuroToolBox Gong Lab\n\nDeep Distance Estimator is trained to learn EDT.\nEDT is the euclidean distance transform and emasures the distance between\ntwo pixels\n\n\na deep distance estimator is trained to learn EDT. Since the Euclidean distance assumes a continuous instead of a Boolean value, pixels that are close to cell boundary have smaller values. Specifically consider the regions with blurred boundaries in the original images.\n\nMUST use conda environment\n'

In [2]:
#Use this when running in google colab
# from google.colab import drive
# drive.mount('/content/gdrive')

In [19]:
import numpy as np
import pandas as pd

import sys
# import cv2
#from skimage.io import imread, imread_collection
from scipy import signal
import scipy.misc
# from scipy.misc import imread
from scipy.ndimage import filters,distance_transform_edt
from matplotlib import pyplot as plt
import imageio # use im = imageio.imread(_____)
import os
from os import listdir

import tensorflow as tf
print(tf.version.VERSION)

import keras
from keras.models import Model
from keras import optimizers
from keras.layers import *

2.0.0


In [20]:

dir_path='/Users/poojap/Documents/ProteinSensorResearch/single_cell_segmentation-master'
main_path=dir_path+'/NMuMg_phase_contrast/training_data'
path_train_input = main_path+'/distance_transform/train/Img'
# no testing file
path_test_input= main_path+'/distance_transform/testImg'
path_train_gt = main_path+'/distance_transform/train/Bwdist'
#no testing file
path_test_gt=main_path+'/distance_transform/testBWdist'

# what is predict?

path_predict= '/content/gdrive/MyDrive/ImageSegmentationFiles/MembraneImages'
# path_predict = '/content/gdrive/MyDrive/single_cell_segmentation-master/NMuMg_phase_contrast/training_data/distance_transform/testImg/crop_phasexy18c2cr1s8.tif'
path_output= '/content/gdrive/MyDrive/single_cell_segmentation-master/output/deepDistanceEstimator'

# This pathway is still wrong
path_frcnn_predict=dir_path+'/frcnn_predict'

nb_epoch =300
batch_size = 10
weight_file= dir_path + '/nmumg_reg_Pooja_5epochs.hdf5'


In [21]:
#As this is a regression problem, train_label should be also normalized.
def prep_data(path_img,path_gt):
    data = []
    label = []

    img_list = sorted(listdir(path_img))
    gt_list=sorted(listdir(path_gt))
    i=0
    while (i<len(img_list)):
        img = np.array(imageio.imread(path_img +'/'+ img_list[i]),dtype=np.float64)
        #img = np.array(imread(path_img +'/'+ img_list[i]))
        #print img.shape
        img=(img-np.amin(img))*1.0/(np.amax(img)-np.amin(img))#img*1.0 transform array to double
        img=img*1.0/np.median(img)
        img_h=img.shape[0]        
        img_w=img.shape[1]        
        img=np.reshape(img,(img_h,img_w,1))         
        data.append(img)  
        
        gt =np.array(imageio.imread(path_gt + '/'+ gt_list[i]))
#---------------with or without normalization-----------------------        
        if np.count_nonzero(gt)!=0:
            nonzero_gt=gt[gt>0]
            gt=gt*1.0/np.median(nonzero_gt)

        gt=np.reshape(gt,(img_h,img_w,1))
        label.append(gt)
        
        i+=1
    data=np.array(data)
    label=np.array(label) 
    print(data.shape, label.shape)
    return data, label     

In [22]:
def prep_prediction_data(path_img):
    data = []
    img_list = sorted(listdir(path_img))
    i=0
    while (i<len(img_list)):
        img = np.array(imageio.imread(path_img +'/'+ img_list[i]),dtype=np.float64)
        img=(img-np.amin(img))*1.0/(np.amax(img)-np.amin(img))#img*1.0 transform array to double
        img=img*1.0/np.median(img)
        img_h=img.shape[0]        
        img_w=img.shape[1]
        img=np.reshape(img,(img_h,img_w,1))
        data.append(img)
        i+=1
    data=np.array(data)
    return data

In [23]:
def conv_block(input_tensor, kernel, filters):
    x=Conv2D(filters, (kernel, kernel), padding='same')(input_tensor)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    return x

In [24]:
kernel = 3
#------------encoder layers--------------------------------
inputs = tf.keras.Input((None,None, 1))
conv1=conv_block(inputs,kernel,filters=64)
conv1=conv_block(conv1,kernel,filters=64)
pool1=MaxPooling2D()(conv1)
    
conv2=conv_block(pool1,kernel,filters=128)
conv2=conv_block(conv2,kernel,filters=128)
pool2=MaxPooling2D()(conv2)

conv3=conv_block(pool2,kernel,filters=256)
conv3=conv_block(conv3,kernel,filters=256)
conv3=conv_block(conv3,kernel,filters=256)
pool3=MaxPooling2D()(conv3)

conv4=conv_block(pool3,kernel,filters=512)
conv4=conv_block(conv4,kernel,filters=512)
conv4=conv_block(conv4,kernel,filters=512)
pool4=MaxPooling2D()(conv4)
 
conv5=conv_block(pool4,kernel,filters=512)
conv5=conv_block(conv5,kernel,filters=512)
conv5=conv_block(conv5,kernel,filters=512)
pool5=MaxPooling2D()(conv5)


#--------------------decoder layers--------------------------

up6=UpSampling2D()(pool5)
conv6=conv_block(up6,kernel,filters=512)
conv6=conv_block(conv6,kernel,filters=512)
conv6=conv_block(conv6,kernel,filters=512)

up7=UpSampling2D()(conv6)
conv7=conv_block(up7,kernel,filters=512)
conv7=conv_block(conv7,kernel,filters=512)
conv7=conv_block(conv7,kernel,filters=512)
    
up8=UpSampling2D()(conv7)
conv8=conv_block(up8,kernel,filters=256)
conv8=conv_block(conv8,kernel,filters=256)
conv8=conv_block(conv8,kernel,filters=256)

up9=UpSampling2D()(conv8)
conv9=conv_block(up9,kernel,filters=128)
conv9=conv_block(conv9,kernel,filters=128)

up10=UpSampling2D()(conv9)
conv10=conv_block(up10,kernel,filters=64)

conv11=conv_block(conv10,kernel=1,filters=1)    
outputs=Activation('relu')(conv11)

autoencoder=Model(inputs=[inputs], outputs=[outputs])

#autoencoder.summary()

autoencoder.compile(loss="mean_squared_error", optimizer='adam', metrics=['mae','acc'])
print ('Compiled: OK')

AttributeError: 'Node' object has no attribute 'output_masks'

In [ ]:
train_data, train_label = prep_data(path_train_input,path_train_gt)

In [ ]:
#---------------------train-------------------------
history=autoencoder.fit(train_data, train_label, batch_size=batch_size, epochs=nb_epoch, verbose=1)
autoencoder.save_weights(weight_file)

In [ ]:
from PIL import Image
from scipy import ndimage
#-------------------predict----------------------------
autoencoder.load_weights(weight_file)
# put htis into the test path same as predict
predict_data=prep_prediction_data(path_predict)
output = autoencoder.predict(predict_data, verbose=0)
print(type(output))
print(output.shape)

#output_label=np.argmax(output[0],axis=-1)
im=output[10][:,:,0]   
np.save(path_output+'/reg_out.npy',im)
data=np.load(path_output+'/reg_out.npy')
print(im.shape)

#save image to 8 bit

imageio.imwrite(path_output+'/reg_out_8bit.png',im)
#save image to the exat value

# img=scipy.misc.toimage(im,high=np.max(im),low=np.min(im),mode='F')
img = Image.fromarray(im, mode = 'F')
img.save(path_output+'/reg_out_exact.tif')
img.save(path_frcnn_predict+'/reg_out_exact.tif')
plt.imshow(im)
plt.axis('off')
plt.savefig(path_output+'/reg_out_300dpi.tif',bbox_inches='tight',format='tif',dpi=300)
plt.show()
